In [327]:
import pyshacl
from rdflib import OWL, RDF, RDFS, SH, Graph, Namespace

In [360]:
file = 'SPARQL-test.ttl'
#file = 'construct_typing.ttl'

g = Graph ()
g.parse(file)
sg = Graph()
sg.parse(file)

# graph before validation
og = Graph()
pyshacl.rdfutil.clone.clone_graph(g, og)

<Graph identifier=Ne8d7926cfbfd4bd2b6744624a09ee97a (<class 'rdflib.graph.Graph'>)>

In [361]:
valid, report_graph, report_text = pyshacl.validate(
    data_graph=g,
    shacl_graph=sg,
    advanced=True,
    js=True,
    allow_warnings=False,
    inplace=True,
    iterate_rules=True,
)

In [362]:
# removing the original graph information from the validated/inferenced graph
added_triples = g - og

In [363]:
added_triples.print()


<http://example.org/testing#instance> a <http://example.org/testing#parameter> .

<http://example.org/testing#instance2> a <http://example.org/testing#variable> .

<http://example.org/testing#instance3> a <http://example.org/testing#NOTvariable> .




In [364]:
namespace_map = {}
for prefix, uriref in report_graph.namespaces():
    namespace_map[prefix] = Namespace(uriref)
    # Need OR for result Message
qs = """
    SELECT ?focusNode ?resultMessage ?resultSeverity ?resultPath
    WHERE {
        BIND(sh:Violation AS ?resultSeverity) .
        ?report rdf:type sh:ValidationReport .
        ?report sh:result ?result .
        ?result sh:focusNode ?focusNode .
        OPTIONAL {?result sh:resultMessage ?resultMessage} .
        ?result sh:resultPath ?resultPath .
        ?result sh:resultSeverity ?resultSeverity .
        }
    """

# pretty colors
color_map = {SH.Violation: 33, SH.Info: 34, SH.Warning: 35}

# query
results = sorted(report_graph.query(qs, initNs=namespace_map))

prev = None
for focusNode, resultMessage,resultSeverity, resultPath in results:
    if focusNode != prev:
        print(focusNode)
        prev = focusNode
    color = color_map[resultSeverity]
    print(f"\x1b[{color}m    {resultMessage}\x1b[0m   {resultPath}\x1b[0m")

http://example.org/testing#instance3
    instance is not a valid variable   http://example.org/testing#test_prop


In [365]:
print(report_text)

Validation Report
Conforms: False
Results (1):
Constraint Violation in HasValueConstraintComponent (http://www.w3.org/ns/shacl#HasValueConstraintComponent):
	Severity: sh:Violation
	Source Shape: test:variable_prop_shape
	Focus Node: test:instance3
	Result Path: test:test_prop
	Message: instance is not a valid variable

